#Exercise 1. Simple mass action reaction system, single compartment
$$2\mathrm{H}^+ + \mathrm{O}^{2-} \longrightarrow \mathrm{H}_2\mathrm{O}$$

In [1]:
from neuron import h, rxd
import time
from pylab import *
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#This is the initialization of the RxD system - should be run only once
h.load_file('stdrun.hoc')                                     #A default NEURON library

comp = h.Section()                                            #Create a compartment called comp
r = rxd.Region([comp])                                        #Create a region that only includes comp
                                                              #(this is where reactions will occur)

hydrogen = rxd.Species(r, name='hydrogen',charge=1,initial=1) #Create H+ species
oxygen = rxd.Species(r, name='oxygen',charge=-2,initial=1)    #Create O2- species
water = rxd.Species(r, name='water',charge=0,initial=0)       #Create H2O species

reaction = rxd.Reaction(2 * hydrogen + oxygen > water, 1.0)   #Reaction 2H + O -> H2O, reaction rate 1.0 (mM)^2/ms

vec_t = h.Vector()                                            #Create vectors for recordings
vec_H = h.Vector()
vec_O = h.Vector()
vec_H2O = h.Vector()
vec_t.record(h._ref_t)
vec_H.record(hydrogen.nodes(comp)(0.5)[0]._ref_concentration) #Record the concentration variables
vec_O.record(oxygen.nodes(comp)(0.5)[0]._ref_concentration)
vec_H2O.record(water.nodes(comp)(0.5)[0]._ref_concentration)

1.0

In [4]:
#This is the main simulation of the RxD system. 
def runneuron(IC_H=1,IC_O=1,IC_H2O=0):
    my_start_time = time.time()
    h.finitialize()                                           #Initialize the dynamic variables
    for node in hydrogen.nodes:                               #Use given initial concentrations (IC_H,
        node.concentration = IC_H                             #  IC_O, IC_H2O) instead of the values
    for node in oxygen.nodes:                                 #  given in the introduction of the species
        node.concentration = IC_O
    for node in water.nodes:
        node.concentration = IC_H2O
    
    h.continuerun(100)                                        #Run for 100 ms

    f,ax = plt.subplots(1,1)
    data_H = array(vec_H)
    data_O = array(vec_O)
    data_H2O = array(vec_H2O)
    data_H[0] = IC_H     # For some reason the first values in the
    data_O[0] = IC_O     # vectors were not correctly initialized
    data_H2O[0] = IC_H2O # and have to be set manually
    ax.plot(array(vec_t),data_H,label='H$^+$')
    ax.plot(array(vec_t),data_O,label='O$^{2-}$')
    ax.plot(array(vec_t),data_H2O,label='H$_2$O')
    ax.legend()
    ax.set_xlabel('$t$ (ms)')
    ax.set_ylabel('$c$ (mmol/l)')
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()

In [5]:
slider = widgets.interact(runneuron, IC_H=widgets.FloatSlider(min=0.0,max=1.0,step=0.01,value=1.0,description='Initial concentration of H+',layout=Layout(width='50%'), continuous_update = False),
                                     IC_O=widgets.FloatSlider(min=0.0,max=1.0,step=0.01,value=1.0,description='Initial concentration of O2-',layout=Layout(width='50%'), continuous_update = False),
                                     IC_H2O=widgets.FloatSlider(min=0.0,max=1.0,step=0.01,value=0.0,description='Initial concentration of H2O',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Initial concentration of H+…

Before answering the following questions, look at the code in the RxD initialization block and see that you understand the steps needed for introducing a reaction at given compartments.
#Question 1: Vary the initial concentrations of H+ and O2-. What is the main difference between cases with low initial H+ and O2- (say, 0.15 mM each) and high H+ and O2- (1.0 mM each)?
#Question 2: Vary the initial concentration of H2O. How does it affect the dynamics of H+ and O2-? Why?